In [15]:
%run /Users/jiamingqu/Desktop/proj/scripts/searching/evaluation.functions.ipynb
%run /Users/jiamingqu/Desktop/proj/scripts/data.modeling/training.tree.model.ipynb
%run /Users/jiamingqu/Desktop/proj/scripts/reranking/significance.testing.ipynb

In [16]:
import pandas as pd
import joblib
import numpy as np

In [17]:
def get_tree_soft_prob_coverage(year,ndocs_rerank=500):
    
    
    # read predited prob.
    df_features_high = pd.read_csv(str(year)+'.high.features.csv',sep=',')
    df_features_high['topicid'] = df_features_high['topicid'].astype(int)
    df_features_high['docid'] = df_features_high['docid'].astype(str)
    
    # features 
    feature_names = ['Human_PM', 'Animal_PM', 'Not_PM', 
                     'Disease_Exact','Disease_General', 'Disease_Specific', 'Disease_Not', 
                     'Gene_Exact','Gene_Missing', 'Gene_Missing_Variant', 'Gene_Diff_Variant',
                     'Demo_Match', 'Demo_Notdiscussed', 'Demo_Exclude']          
                                   
    
    all_prob = []
    
    for topic in [*range(1,51)]:
        
        # read ground truth
        answer = read_answers(year,topic)

        # 500 results under a topic
        df_topic = df_features_high.loc[df_features_high.topicid == topic]
        df_topic = df_topic.head(500)

        # we only rerank the first n results
        df_topic_reanking = df_topic.head(ndocs_rerank)
        df_topic_noreanking = df_topic.tail(500-ndocs_rerank)

        for index,rows in df_topic_reanking.iterrows():

            docid = str(df_topic_reanking.loc[index,"docid"])
            features = df_topic_reanking.loc[index, feature_names]
            results = probalistic_tree_balanced_all_probs(dict(features))
            results.sort(reverse=True)
        
        sum_prob_top3 = results[0]+results[1]+results[2]

    all_prob.append(sum_prob_top3)
    
    print(np.mean(all_prob))

In [18]:
get_tree_soft_prob_coverage(2018)

0.6201435901114503
